# Modelo para testar a hipótese "Períodos festivos, épocas do ano (estações do ano) e período de férias (julho, agosto, dezembro e janeiro) tem impacto sobre o número semanal de casos."

## Import das bibliotecas e dos dados

In [54]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import statsmodels.api as sm
import statsmodels.formula.api as smf
import linearmodels as lm

pd.set_option('display.max_columns', None)

path = os.path.abspath('../data/analise_crime_filtrado.csv')
df_guarda = pd.read_csv(path,comment='#', low_memory=False, parse_dates=['OCORRENCIA_DATA'])

path = os.path.abspath('../data/clima.csv')
df_clima = pd.read_csv(path,comment='#', low_memory=False)

df = pd.merge(df_guarda, df_clima, how='left')

df.head()

,OCORRENCIA_CODIGO,NATUREZA1_DESCRICAO,NATUREZA,ATENDIMENTO_BAIRRO_NOME,REGIONAL_FATO_NOME,OCORRENCIA_ANO,OCORRENCIA_DATA,OCORRENCIA_HORA,OCORRENCIA_HORA_SEM_MINUTO,OCORRENCIA_MES,OCORRENCIA_DIA_SEMANA,OCORRENCIA_DATA_SEM_HORARIO,FERIADO,LATITUDE,LONGITUDE,T_MEDIA,T_MINIMA,T_MAXIMA,CHUVA
0,5122.0,Roubo,ROUBO,XAXIM,BOQUEIRÃO,2009.0,2009-03-13 09:20:00,09:20:00,9,3.0,SEXTA,2009-03-13,0.0,-25.426683,-49.272357,19.8,17.1,23.8,137
1,5507.0,Atos obscenos/libidinosos,PUDOR,BAIRRO ALTO,BOA VISTA,2009.0,2009-03-19 12:05:00,12:05:00,12,3.0,QUINTA,2009-03-19,0.0,-25.412337,-49.205083,19.8,17.1,23.8,137
2,4899.0,Embriaguez,RISCO_VIDA,FAZENDINHA,PORTÃO,2009.0,2009-03-09 14:00:00,14:00:00,14,3.0,SEGUNDA,2009-03-09,0.0,NaN,NaN,19.8,17.1,23.8,137
3,5401.0,Furto,ROUBO,FAZENDINHA,PORTÃO,2009.0,2009-03-16 20:11:00,20:11:00,20,3.0,SEGUNDA,2009-03-16,0.0,NaN,NaN,19.8,17.1,23.8,137
4,3084.0,Ameaça,AGRESSAO,BOA VISTA,BOA VISTA,2009.0,2009-02-13 08:10:00,08:10:00,8,2.0,SEXTA,2009-02-13,0.0,-25.393377,-49.259794,20.6,17.8,24.7,199


## Agrupando e filtrando dados de interesse

In [27]:
df2 = df.groupby(['OCORRENCIA_DATA_SEM_HORARIO'])['OCORRENCIA_DATA_SEM_HORARIO'].count().reset_index(name = 'CASOS')

df3 = df.groupby(['OCORRENCIA_DATA_SEM_HORARIO']).mean().reset_index()[['OCORRENCIA_ANO', 'OCORRENCIA_MES', 'OCORRENCIA_DATA_SEM_HORARIO', 'FERIADO', 'T_MEDIA', 'CHUVA']]

df4 = df.groupby(['OCORRENCIA_DATA_SEM_HORARIO']).max().reset_index()[['OCORRENCIA_DATA_SEM_HORARIO','OCORRENCIA_DIA_SEMANA']]

df3 = pd.merge(df2, df3, how='left')

df3 = pd.merge(df3, df4, how='left')

df3 = df3.astype({'OCORRENCIA_ANO': int, 'OCORRENCIA_MES': int, 'FERIADO': int, 'CHUVA': int})

df2 = df3[['OCORRENCIA_DATA_SEM_HORARIO', 'OCORRENCIA_ANO', 'OCORRENCIA_MES','OCORRENCIA_DIA_SEMANA', 'CASOS', 'FERIADO', 'T_MEDIA', 'CHUVA', ]]

df2.columns = ['DATA', 'ANO', 'MES', 'DIA_SEMANA', 'CASOS', 'FERIADO', 'TEMPERATURA_MEDIA', 'CHUVA']

df2 = df2.sort_values('DATA')

df2.loc[df2["DIA_SEMANA"] == "SEGUNDA", "DIA_SEMANA"] = 1
df2.loc[df2["DIA_SEMANA"] == "TERÇA", "DIA_SEMANA"] = 2
df2.loc[df2["DIA_SEMANA"] == "QUARTA", "DIA_SEMANA"] = 3
df2.loc[df2["DIA_SEMANA"] == "QUINTA", "DIA_SEMANA"] = 4
df2.loc[df2["DIA_SEMANA"] == "SEXTA", "DIA_SEMANA"] = 5
df2.loc[df2["DIA_SEMANA"] == "SÁBADO", "DIA_SEMANA"] = 6
df2.loc[df2["DIA_SEMANA"] == "DOMINGO", "DIA_SEMANA"] = 7

df2 = df2.astype({'DIA_SEMANA': int})

df2.head()

,DATA,ANO,MES,DIA_SEMANA,CASOS,FERIADO,TEMPERATURA_MEDIA,CHUVA
0,2009-01-01,2009,1,4,2,1,20.4,233
1,2009-01-02,2009,1,5,4,0,20.4,233
2,2009-01-03,2009,1,6,3,0,20.4,233
3,2009-01-04,2009,1,7,9,0,20.4,233
4,2009-01-05,2009,1,1,7,0,20.4,233


# Pooled OLS

In [36]:
model = smf.ols("CASOS ~ FERIADO + TEMPERATURA_MEDIA + CHUVA", data=df2)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.611
Date:                Wed, 07 Dec 2022   Prob (F-statistic):             0.0337
Time:                        22:38:57   Log-Likelihood:                -15027.
No. Observations:                4793   AIC:                         3.006e+04
Df Residuals:                    4788   BIC:                         3.010e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.9224      0.681     14.574      0.000       8.588      11.257
MES                  -0.0281      0.024     -1.154      0.248      -0.076       0.020
FERIADO              -0.2351      0.453     -0.519      0.604      -1.122       0.652
TEMPERATURA_MEDIA     0.1396      0.049      2.846      0.004       0.043       0.236
CHUVA                -0.0068      0.003     -2.458      0.014      -0.012      -0.001
==============================================================================
Omnibus:                      456.596   Durbin-Watson:                   0.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              765.165
Skew:                           0.682   Prob(JB):                    7.02e-167
Kurtosis:                       4.403   Cond. No.                     1.23e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Efeitos Fixos

In [38]:
model = smf.ols("CASOS ~ FERIADO + TEMPERATURA_MEDIA + CHUVA + C(ANO)", data=df2)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     272.3
Date:                Wed, 07 Dec 2022   Prob (F-statistic):               0.00
Time:                        22:42:20   Log-Likelihood:                -13478.
No. Observations:                4793   AIC:                         2.699e+04
Df Residuals:                    4776   BIC:                         2.710e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             3.6319      0.490      7.419      0.000       2.672       4.592
C(ANO)[T.2010]        0.2166      0.301      0.721      0.471      -0.373       0.806
C(ANO)[T.2011]        1.1771      0.300      3.928      0.000       0.590       1.765
C(ANO)[T.2012]        2.2564      0.299      7.550      0.000       1.670       2.842
C(ANO)[T.2013]        3.7487      0.299     12.517      0.000       3.162       4.336
C(ANO)[T.2014]        8.0435      0.299     26.895      0.000       7.457       8.630
C(ANO)[T.2015]       10.9558      0.299     36.633      0.000      10.369      11.542
C(ANO)[T.2016]        9.4913      0.299     31.758      0.000       8.905      10.077
C(ANO)[T.2017]        8.4818      0.299     28.361      0.000       7.896       9.068
C(ANO)[T.2018]        7.9257      0.299     26.501      0.000       7.339       8.512
C(ANO)[T.2019]        8.2900      0.299     27.720      0.000       7.704       8.876
C(ANO)[T.2020]       10.4995      0.299     35.132      0.000       9.914      11.085
C(ANO)[T.2021]        8.1613      0.299     27.289      0.000       7.575       8.748
C(ANO)[T.2022]        9.6087      0.577     16.667      0.000       8.478      10.739
FERIADO              -0.2528      0.328     -0.771      0.441      -0.896       0.390
TEMPERATURA_MEDIA     0.1499      0.035      4.243      0.000       0.081       0.219
CHUVA                -0.0084      0.002     -4.194      0.000      -0.012      -0.004
==============================================================================
Omnibus:                      904.817   Durbin-Watson:                   1.758
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4035.668
Skew:                           0.853   Prob(JB):                         0.00
Kurtosis:                       7.159   Cond. No.                     2.09e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
model = smf.ols("CASOS ~ FERIADO + TEMPERATURA_MEDIA + CHUVA + C(ANO) + C(DIA_SEMANA)", data=df2)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.485
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     204.5
Date:                Wed, 07 Dec 2022   Prob (F-statistic):               0.00
Time:                        23:03:31   Log-Likelihood:                -13440.
No. Observations:                4793   AIC:                         2.693e+04
Df Residuals:                    4770   BIC:                         2.707e+04
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              2.9297      0.506      5.789      0.000       1.938       3.922
C(ANO)[T.2010]         0.2214      0.298      0.742      0.458      -0.363       0.806
C(ANO)[T.2011]         1.1791      0.297      3.964      0.000       0.596       1.762
C(ANO)[T.2012]         2.2576      0.297      7.611      0.000       1.676       2.839
C(ANO)[T.2013]         3.7530      0.297     12.625      0.000       3.170       4.336
C(ANO)[T.2014]         8.0455      0.297     27.103      0.000       7.463       8.627
C(ANO)[T.2015]        10.9574      0.297     36.912      0.000      10.375      11.539
C(ANO)[T.2016]         9.4905      0.297     31.993      0.000       8.909      10.072
C(ANO)[T.2017]         8.4812      0.297     28.571      0.000       7.899       9.063
C(ANO)[T.2018]         7.9289      0.297     26.710      0.000       7.347       8.511
C(ANO)[T.2019]         8.2927      0.297     27.936      0.000       7.711       8.875
C(ANO)[T.2020]        10.5018      0.297     35.402      0.000       9.920      11.083
C(ANO)[T.2021]         8.1624      0.297     27.497      0.000       7.580       8.744
C(ANO)[T.2022]         9.5990      0.572     16.774      0.000       8.477      10.721
C(DIA_SEMANA)[T.2]     0.2282      0.216      1.054      0.292      -0.196       0.652
C(DIA_SEMANA)[T.3]     0.4600      0.216      2.126      0.034       0.036       0.884
C(DIA_SEMANA)[T.4]     0.6123      0.216      2.829      0.005       0.188       1.037
C(DIA_SEMANA)[T.5]     0.7815      0.216      3.617      0.000       0.358       1.205
C(DIA_SEMANA)[T.6]     1.4166      0.216      6.548      0.000       0.993       1.841
C(DIA_SEMANA)[T.7]     1.4233      0.217      6.574      0.000       0.999       1.848
FERIADO               -0.1399      0.326     -0.429      0.668      -0.779       0.500
TEMPERATURA_MEDIA      0.1496      0.035      4.266      0.000       0.081       0.218
CHUVA                 -0.0084      0.002     -4.228      0.000      -0.012      -0.005
==============================================================================
Omnibus:                      843.597   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3607.974
Skew:                           0.804   Prob(JB):                         0.00
Kurtosis:                       6.934   Cond. No.                     2.09e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
df3 = df.groupby(['REGIONAL_FATO_NOME', 'OCORRENCIA_ANO', 'OCORRENCIA_MES']).size().reset_index(name = 'CASOS')

df3 = pd.merge(df3, df_clima, how='left')

df3

,REGIONAL_FATO_NOME,OCORRENCIA_ANO,OCORRENCIA_MES,CASOS,T_MEDIA,T_MINIMA,T_MAXIMA,CHUVA
0,BAIRRO NOVO,2009.0,1.0,19,20.4,17.6,24.6,233
1,BAIRRO NOVO,2009.0,2.0,19,20.6,17.8,24.7,199
2,BAIRRO NOVO,2009.0,3.0,10,19.8,17.1,23.8,137
3,BAIRRO NOVO,2009.0,4.0,12,18.1,15.2,22.3,84
4,BAIRRO NOVO,2009.0,5.0,5,15.0,12.0,18.9,100
...,...,...,...,...,...,...,...,...
1510,TATUQUARA,2021.0,10.0,15,17.4,14.0,22.2,142
1511,TATUQUARA,2021.0,11.0,24,18.1,15.0,22.6,134
1512,TATUQUARA,2021.0,12.0,19,19.8,16.6,24.2,168
1513,TATUQUARA,2022.0,1.0,22,20.4,17.6,24.6,233


In [60]:
model = smf.ols("CASOS ~ OCORRENCIA_MES + T_MEDIA + CHUVA", data=df3)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2797
Date:                Wed, 07 Dec 2022   Prob (F-statistic):              0.840
Time:                        23:21:06   Log-Likelihood:                -7363.9
No. Observations:                1515   AIC:                         1.474e+04
Df Residuals:                    1511   BIC:                         1.476e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         33.5297      6.839      4.902      0.000      20.114      46.945
OCORRENCIA_MES    -0.0268      0.244     -0.110      0.912      -0.504       0.451
T_MEDIA            0.3233      0.491      0.658      0.510      -0.640       1.287
CHUVA             -0.0253      0.028     -0.916      0.360      -0.079       0.029
==============================================================================
Omnibus:                      769.075   Durbin-Watson:                   0.129
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4065.674
Skew:                           2.429   Prob(JB):                         0.00
Kurtosis:                       9.388   Cond. No.                     1.24e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [61]:
model = smf.ols("CASOS ~ T_MEDIA + CHUVA + C(REGIONAL_FATO_NOME)", data=df3)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.680
Model:                            OLS   Adj. R-squared:                  0.678
Method:                 Least Squares   F-statistic:                     266.1
Date:                Wed, 07 Dec 2022   Prob (F-statistic):               0.00
Time:                        23:22:21   Log-Likelihood:                -6501.0
No. Observations:                1515   AIC:                         1.303e+04
Df Residuals:                    1502   BIC:                         1.310e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                        22.1717      3.735      5.936      0.000      14.846      29.498
C(REGIONAL_FATO_NOME)[T.BOA VISTA]                3.8608      1.997      1.933      0.053      -0.057       7.778
C(REGIONAL_FATO_NOME)[T.BOQUEIRÃO]               12.5063      1.997      6.262      0.000       8.589      16.424
C(REGIONAL_FATO_NOME)[T.CAJURU]                   2.9684      1.997      1.486      0.137      -0.949       6.886
C(REGIONAL_FATO_NOME)[T.CIC]                     -4.7785      1.997     -2.393      0.017      -8.696      -0.861
C(REGIONAL_FATO_NOME)[T.MATRIZ]                  82.7911      1.997     41.454      0.000      78.874      86.709
C(REGIONAL_FATO_NOME)[T.PINHEIRINHO]             -0.7722      1.997     -0.387      0.699      -4.690       3.145
C(REGIONAL_FATO_NOME)[T.PORTÃO]                   9.8544      1.997      4.934      0.000       5.937      13.772
C(REGIONAL_FATO_NOME)[T.REGIÃO METROPOLITANA]   -24.3187      4.315     -5.636      0.000     -32.783     -15.854
C(REGIONAL_FATO_NOME)[T.SANTA FELICIDADE]        -4.1835      1.997     -2.095      0.036      -8.101      -0.266
C(REGIONAL_FATO_NOME)[T.TATUQUARA]              -10.3750      2.501     -4.149      0.000     -15.280      -5.470
T_MEDIA                                           0.3804      0.277      1.374      0.170      -0.163       0.923
CHUVA                                            -0.0228      0.016     -1.463      0.144      -0.053       0.008
==============================================================================
Omnibus:                      150.867   Durbin-Watson:                   0.379
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1088.861
Skew:                          -0.070   Prob(JB):                    3.60e-237
Kurtosis:                       7.151   Cond. No.                     1.68e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.68e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [62]:
model = smf.ols("CASOS ~ T_MEDIA + CHUVA + C(REGIONAL_FATO_NOME) + C(OCORRENCIA_ANO)", data=df3)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  CASOS   R-squared:                       0.830
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     291.0
Date:                Wed, 07 Dec 2022   Prob (F-statistic):               0.00
Time:                        23:22:54   Log-Likelihood:                -6021.6
No. Observations:                1515   AIC:                         1.210e+04
Df Residuals:                    1489   BIC:                         1.223e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         2.6989      2.989      0.903      0.367      -3.165       8.563
C(REGIONAL_FATO_NOME)[T.BOA VISTA]                3.8608      1.462      2.641      0.008       0.994       6.728
C(REGIONAL_FATO_NOME)[T.BOQUEIRÃO]               12.5063      1.462      8.556      0.000       9.639      15.374
C(REGIONAL_FATO_NOME)[T.CAJURU]                   2.9684      1.462      2.031      0.042       0.101       5.836
C(REGIONAL_FATO_NOME)[T.CIC]                     -4.7785      1.462     -3.269      0.001      -7.646      -1.911
C(REGIONAL_FATO_NOME)[T.MATRIZ]                  82.7911      1.462     56.639      0.000      79.924      85.658
C(REGIONAL_FATO_NOME)[T.PINHEIRINHO]             -0.7722      1.462     -0.528      0.597      -3.639       2.095
C(REGIONAL_FATO_NOME)[T.PORTÃO]                   9.8544      1.462      6.742      0.000       6.987      12.722
C(REGIONAL_FATO_NOME)[T.REGIÃO METROPOLITANA]   -35.2193      3.185    -11.058      0.000     -41.467     -28.972
C(REGIONAL_FATO_NOME)[T.SANTA FELICIDADE]        -4.1835      1.462     -2.862      0.004      -7.051      -1.316
C(REGIONAL_FATO_NOME)[T.TATUQUARA]              -18.6346      1.866     -9.985      0.000     -22.295     -14.974
C(OCORRENCIA_ANO)[T.2010.0]                       0.4907      1.768      0.278      0.781      -2.977       3.959
C(OCORRENCIA_ANO)[T.2011.0]                       3.8981      1.768      2.205      0.028       0.430       7.366
C(OCORRENCIA_ANO)[T.2012.0]                       7.7870      1.768      4.404      0.000       4.319      11.255
C(OCORRENCIA_ANO)[T.2013.0]                      12.6019      1.768      7.128      0.000       9.134      16.070
C(OCORRENCIA_ANO)[T.2014.0]                      27.2778      1.768     15.429      0.000      23.810      30.746
C(OCORRENCIA_ANO)[T.2015.0]                      36.9085      1.758     20.996      0.000      33.460      40.357
C(OCORRENCIA_ANO)[T.2016.0]                      30.3562      1.720     17.648      0.000      26.982      33.730
C(OCORRENCIA_ANO)[T.2017.0]                      27.2075      1.725     15.776      0.000      23.825      30.590
C(OCORRENCIA_ANO)[T.2018.0]                      25.5360      1.727     14.783      0.000      22.148      28.924
C(OCORRENCIA_ANO)[T.2019.0]                      26.6658      1.722     15.484      0.000      23.288      30.044
C(OCORRENCIA_ANO)[T.2020.0]                      33.3076      1.718     19.383      0.000      29.937      36.678
C(OCORRENCIA_ANO)[T.2021.0]                      26.2069      1.731     15.144      0.000      22.812      29.602
C(OCORRENCIA_ANO)[T.2022.0]                      29.7458      3.165      9.397      0.000      23.537      35.955
T_MED

In [71]:
df4 = df.groupby(['REGIONAL_FATO_NOME', 'OCORRENCIA_DATA_SEM_HORARIO']).size().reset_index(name = 'CASOS')

df5 = df.groupby(['OCORRENCIA_DATA_SEM_HORARIO']).mean().reset_index()[['OCORRENCIA_ANO', 'OCORRENCIA_MES', 'OCORRENCIA_DATA_SEM_HORARIO', 'FERIADO', 'T_MEDIA', 'CHUVA']]

df4 = pd.merge(df4, df5, how='left')

df4['OCORRENCIA_DATA_SEM_HORARIO'] = pd.to_datetime(df4['OCORRENCIA_DATA_SEM_HORARIO'])

df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26860 entries, 0 to 26859
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   REGIONAL_FATO_NOME           26860 non-null  object        
 1   OCORRENCIA_DATA_SEM_HORARIO  26860 non-null  datetime64[ns]
 2   CASOS                        26860 non-null  int64         
 3   OCORRENCIA_ANO               26860 non-null  float64       
 4   OCORRENCIA_MES               26860 non-null  float64       
 5   FERIADO                      26860 non-null  float64       
 6   T_MEDIA                      26860 non-null  float64       
 7   CHUVA                        26860 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 1.8+ MB


In [72]:
df_fe = df4.set_index(["REGIONAL_FATO_NOME", "OCORRENCIA_DATA_SEM_HORARIO"])

df_fe

CASOS  OCORRENCIA_ANO  \
REGIONAL_FATO_NOME OCORRENCIA_DATA_SEM_HORARIO                          
BAIRRO NOVO        2009-01-02                       1          2009.0   
                   2009-01-04                       2          2009.0   
                   2009-01-09                       1          2009.0   
                   2009-01-12                       1          2009.0   
                   2009-01-13                       2          2009.0   
...                                               ...             ...   
TATUQUARA          2022-02-21                       1          2022.0   
                   2022-02-22                       1          2022.0   
                   2022-02-25                       2          2022.0   
                   2022-02-26                       2          2022.0   
                   2022-02-27                       2          2022.0   

                                                OCORRENCIA_MES  FERIADO  \
REGIONAL_FATO_NOME OCORRENCIA_DATA_SEM_HORARIO                            
BAIRRO NOVO        2009-01-02                              1.0      0.0   
                   2009-01-04                              1.0      0.0   
                   2009-01-09                              1.0      0.0   
                   2009-01-12                              1.0      0.0   
                   2009-01-13                              1.0      0.0   
...                                                        ...      ...   
TATUQUARA          2022-02-21                              2.0      0.0   
                   2022-02-22                              2.0      0.0   
                   2022-02-25                              2.0      0.0   
                   2022-02-26                              2.0      0.0   
                   2022-02-27                              2.0      0.0   

                                                T_MEDIA  CHUVA  
REGIONAL_FATO_NOME OCORRENCIA_DATA_SEM_HORARIO                  
BAIRRO NOVO        2009-01-02                      20.4  233.0  
                   2009-01-04                      20.4  233.0  
                   2009-01-09                      20.4  233.0  
                   2009-01-12                      20.4  233.0  
                   2009-01-13                      20.4  233.0  
...                                                 ...    ...  
TATUQUARA          2022-02-21                      20.6  199.0  
                   2022-02-22                      20.6  199.0  
                   2022-02-25                      20.6  199.0  
                   2022-02-26                      20.6  199.0  
                   2022-02-27                      20.6  199.0  

[26860 rows x 6 columns]

In [73]:
exog_vars = ["T_MEDIA", "CHUVA"]
exog = sm.add_constant(df_fe[exog_vars])
endog = df_fe["CASOS"]
mod_entity = lm.PanelOLS(endog, exog, entity_effects=True, time_effects=False)
res_entity = mod_entity.fit()
print(res_entity)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  CASOS   R-squared:                        0.0001
Estimator:                   PanelOLS   R-squared (Between):             -0.1256
No. Observations:               26860   R-squared (Within):               0.0001
Date:                Wed, Dec 07 2022   R-squared (Overall):          -2.182e-05
Time:                        23:37:39   Log-likelihood                -4.464e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      1.3642
Entities:                          11   P-value                           0.2556
Avg Obs:                       2441.8   Distribution:                 F(2,26847)
Min Obs:                       22.000                                           
Max Obs:                       4466.0   F-statistic (robust):             1.3642
                            

In [74]:
exog_vars = ["T_MEDIA", "CHUVA"]
exog = sm.add_constant(df_fe[exog_vars])
endog = df_fe["CASOS"]
mod_entity_time = lm.PanelOLS(endog, exog, entity_effects=True, time_effects=True)
res_entity_time = mod_entity_time.fit()
print(res_entity_time)

AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          const, T_MEDIA, CHUVA
          const, T_MEDIA, CHUVA

Set drop_absorbed=True to automatically drop absorbed variables.
